In [0]:
# Add widgets for parameters
dbutils.widgets.text("source_path", "/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/events_oct")
dbutils.widgets.dropdown("layer", "bronze", ["bronze","silver","gold"])

In [0]:
# Use parameters
source = dbutils.widgets.get("source_path")
layer = dbutils.widgets.get("layer")

In [0]:
print(source,"-",layer)

/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/events_oct - bronze


In [0]:
from pyspark.sql import functions as F

def run_layer(layer_name):
    # BRONZE: Raw ingestion
    if layer_name == "bronze":
        path = source
        raw = spark.read.format("delta").load(path)
        raw.withColumn("ingestion_ts", F.current_timestamp()) \
            .write.format("delta").mode("overwrite")\
            .save("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/All_layers/bronze/events")
    # SILVER: Cleaned data
    elif layer_name == "silver":
        bronze = spark.read.format("delta")\
        .load("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/All_layers/bronze/events")
        silver = bronze.filter(F.col("price") > 0) \
            .filter(F.col("price") < 10000) \
            .dropDuplicates(["user_session", "event_time"]) \
            .withColumn("event_date", F.to_date("event_time")) \
            .withColumn("price_tier",
                        F.when(F.col("price") < 10, "budget").when(F.col("price") < 50, "mid")
                        .otherwise("premium"))
        silver.write.format("delta")\
            .mode("overwrite").save("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/All_layers/silver/events")
     # GOLD: Aggregated data
    elif layer_name == "gold":
        silver = spark.read.format("delta")\
            .load("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/All_layers/silver/events")
        product_perf = silver.groupBy("product_id") \
                .agg(F.countDistinct(F.when(F.col("event_type")=="view", "user_id")).alias("views"),
                     F.countDistinct(F.when(F.col("event_type")=="purchase", "user_id")).alias("purchases"),
                     F.sum(F.when(F.col("event_type")=="purchase", F.col("price"))).alias("revenue")
                     ).withColumn("conversion_rate", F.try_divide(F.col("purchases"),F.col("views"))*100)
        product_perf.write.format("delta").mode("overwrite")\
            .save("/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/All_layers/gold/events")

In [0]:
run_layer(layer)

In [0]:
%sql
select * from
delta.`/Volumes/workspace/ecommerce_idc/ecommerce_data_idc/delta/All_layers/gold/events`

product_id,views,purchases,revenue,conversion_rate
5701087,1,1,5245.849999999999,100.0
1005159,1,1,763047.4699999999,100.0
8500290,1,1,14674.03,100.0
45300001,1,1,561.14,100.0
17300014,1,1,1087.3200000000004,100.0
29502246,1,1,451.7,100.0
6902812,1,1,647.6100000000001,100.0
9800341,1,1,337.16999999999996,100.0
7004004,1,1,2959.6299999999997,100.0
28714643,1,1,627.0400000000001,100.0
